In [ ]:
# activate virtual environment
# pip install requests
# pip install pymysql
# pip install bs4
# pip install lxml


In [2]:
import requests
from bs4 import BeautifulSoup
import pymysql

In [3]:
base_url = 'https://turbo.az/autos'

In [4]:
response = requests.get(base_url)

In [5]:
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.content, 'lxml')

In [7]:
page_content = soup.find('div', class_='products-container')

In [8]:
all_ = page_content.find_all('div', class_='tz-container')

In [9]:
elanlar = all_[4]

In [10]:
elanlar

<div class="tz-container"><div class="products-title"><p class="products-title-amount">10000 elan</p><div class="products-title-sort"><select id="sort-variants" name="sort-variants"><option data-path="/autos?q%5Bsort%5D=created_at" value="created_at">Tarixə görə</option>
<option data-path="/autos?q%5Bsort%5D=price_asc" value="price_asc">Əvvəlcə ucuz</option>
<option data-path="/autos?q%5Bsort%5D=price_desc" value="price_desc">Əvvəlcə bahalı</option>
<option data-path="/autos?q%5Bsort%5D=mileage" value="mileage">Yürüş</option>
<option data-path="/autos?q%5Bsort%5D=reg_year" value="reg_year">Buraxılış ili</option></select></div></div><div class="products"><div class="products-i"><a class="products-i__link" href="/autos/6368708-lada-vaz-niva" target="_blank"></a><a class="js-bookmark-item-6368708" data-method="post" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking"></div></a><a class="hide js-unbookmark-item-6368708" data-method="dele

In [11]:
items = elanlar.find_all('div', class_='products-i')

In [12]:
len(items)

24

In [13]:
items[0]

<div class="products-i"><a class="products-i__link" href="/autos/6368708-lada-vaz-niva" target="_blank"></a><a class="js-bookmark-item-6368708" data-method="post" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking"></div></a><a class="hide js-unbookmark-item-6368708" data-method="delete" data-remote="true" href="/autos/6368708-lada-vaz-niva/bookmarks" rel="nofollow"><div class="bookmarking added"></div></a><div class="products-i__top"><img alt="LADA (VAZ) Niva" loading="lazy" src="https://turbo.azstatic.com/uploads/f460x343/2022%2F06%2F09%2F18%2F59%2F01%2F06858b4c-5af5-456a-b0ab-80a20d7353f1%2F85114_Zz7YXmUQgX0_U_O9wEIaNQ.jpg"/><div class="products-i__info"></div></div><div class="products-i__bottom"><div class="products-i__price products-i__bottom-text"><div class="product-price">8 500 <span>AZN</span></div></div><div class="products-i__name products-i__bottom-text">LADA (VAZ) Niva</div><div class="products-i__attributes products-i_

In [14]:
for item in items:
    item_url = 'https://turbo.az' + item.find('a').get('href')
    details = item.find('div', class_='products-i__bottom')

    marka_model = details.find('div', class_='products-i__name products-i__bottom-text').text
    price = details.find('div', class_='product-price').text
    
    print(item_url, marka_model, price)
    print('===' * 30)

https://turbo.az/autos/6368708-lada-vaz-niva LADA (VAZ) Niva 8 500 AZN
https://turbo.az/autos/6355461-opel-astra Opel Astra 12 700 AZN
https://turbo.az/autos/6368704-mercedes-cls-350 Mercedes CLS 350 19 000 AZN
https://turbo.az/autos/6368703-lada-vaz-2115 LADA (VAZ) 2115 7 000 AZN
https://turbo.az/autos/6312031-chevrolet-trax Chevrolet Trax 24 800 AZN
https://turbo.az/autos/6357254-kia-optima Kia Optima 31 500 AZN
https://turbo.az/autos/6368701-daewoo-gentra Daewoo Gentra 9 000 AZN
https://turbo.az/autos/6368700-opel-zafira Opel Zafira 9 800 AZN
https://turbo.az/autos/6368712-toyota-rav-4 Toyota RAV 4 32 300 $
https://turbo.az/autos/6368699-ford-fusion Ford Fusion 19 000 AZN
https://turbo.az/autos/6355765-mitsubishi-mirage Mitsubishi Mirage 12 900 AZN
https://turbo.az/autos/6324514-bmw-320 BMW 320 15 800 AZN
https://turbo.az/autos/6360229-opel-astra Opel Astra 12 900 AZN
https://turbo.az/autos/6368682-iveco-astra-hd8 Iveco Astra HD8 34 999 AZN
https://turbo.az/autos/6368695-tofas-sahin

In [15]:
connection = pymysql.connect(
    host='localhost', # 127.0.0.1
    user='root',
    password ='mysql123',
    database='turbo_db'
)

In [16]:
create_table = """
create table cars (
    id int(11) not null auto_increment,
    url varchar(400) not null,
    price varchar(100),
    marka_model varchar(200),
    primary key (id)
)
"""

In [17]:
with connection.cursor() as cursor:
    cursor.execute(create_table)

In [18]:
for item in items:
    item_url = 'https://turbo.az' + item.find('a').get('href')
    
    details = item.find('div', class_='products-i__bottom')

    marka_model = details.find('div', class_='products-i__name products-i__bottom-text').text
    price = details.find('div', class_='product-price').text
    
    insert_sql = 'insert into cars (url, price, marka_model) values(%s, %s, %s)'
    values = (item_url, price, marka_model)
    
    with connection.cursor() as cursor:
        cursor.execute(insert_sql, values)
        connection.commit()

### Inner parsing parsing

In [23]:
select_sql = 'select * from cars'
with connection.cursor() as cursor:
    cursor.execute(select_sql)

In [24]:
cursor.fetchone()

(1,
 'https://turbo.az/autos/6368708-lada-vaz-niva',
 '8 500 AZN',
 'LADA (VAZ) Niva')

In [25]:
cursor.fetchall()

((2, 'https://turbo.az/autos/6355461-opel-astra', '12 700 AZN', 'Opel Astra'),
 (3,
  'https://turbo.az/autos/6368704-mercedes-cls-350',
  '19 000 AZN',
  'Mercedes CLS 350'),
 (4,
  'https://turbo.az/autos/6368703-lada-vaz-2115',
  '7 000 AZN',
  'LADA (VAZ) 2115'),
 (5,
  'https://turbo.az/autos/6312031-chevrolet-trax',
  '24 800 AZN',
  'Chevrolet Trax'),
 (6, 'https://turbo.az/autos/6357254-kia-optima', '31 500 AZN', 'Kia Optima'),
 (7,
  'https://turbo.az/autos/6368701-daewoo-gentra',
  '9 000 AZN',
  'Daewoo Gentra'),
 (8, 'https://turbo.az/autos/6368700-opel-zafira', '9 800 AZN', 'Opel Zafira'),
 (9,
  'https://turbo.az/autos/6368712-toyota-rav-4',
  '32 300 $',
  'Toyota RAV 4'),
 (10,
  'https://turbo.az/autos/6368699-ford-fusion',
  '19 000 AZN',
  'Ford Fusion'),
 (11,
  'https://turbo.az/autos/6355765-mitsubishi-mirage',
  '12 900 AZN',
  'Mitsubishi Mirage'),
 (12, 'https://turbo.az/autos/6324514-bmw-320', '15 800 AZN', 'BMW 320'),
 (13, 'https://turbo.az/autos/6360229-ope

In [26]:
connection = pymysql.connect(
    host='localhost', # 127.0.0.1
    user='root',
    password ='mysql123',
    database='turbo_db',
    cursorclass=pymysql.cursors.DictCursor
)

In [79]:
select_sql = 'select id, url from cars'
with connection.cursor() as cursor:
    cursor.execute(select_sql)

In [80]:
items = cursor.fetchall()

In [81]:
items

[{'id': 1, 'url': 'https://turbo.az/autos/6368708-lada-vaz-niva'},
 {'id': 2, 'url': 'https://turbo.az/autos/6355461-opel-astra'},
 {'id': 3, 'url': 'https://turbo.az/autos/6368704-mercedes-cls-350'},
 {'id': 4, 'url': 'https://turbo.az/autos/6368703-lada-vaz-2115'},
 {'id': 5, 'url': 'https://turbo.az/autos/6312031-chevrolet-trax'},
 {'id': 6, 'url': 'https://turbo.az/autos/6357254-kia-optima'},
 {'id': 7, 'url': 'https://turbo.az/autos/6368701-daewoo-gentra'},
 {'id': 8, 'url': 'https://turbo.az/autos/6368700-opel-zafira'},
 {'id': 9, 'url': 'https://turbo.az/autos/6368712-toyota-rav-4'},
 {'id': 10, 'url': 'https://turbo.az/autos/6368699-ford-fusion'},
 {'id': 11, 'url': 'https://turbo.az/autos/6355765-mitsubishi-mirage'},
 {'id': 12, 'url': 'https://turbo.az/autos/6324514-bmw-320'},
 {'id': 13, 'url': 'https://turbo.az/autos/6360229-opel-astra'},
 {'id': 14, 'url': 'https://turbo.az/autos/6368682-iveco-astra-hd8'},
 {'id': 15, 'url': 'https://turbo.az/autos/6368695-tofas-sahin'},
 

In [ ]:
# {'Ban növü': 'Offroader / SUV',
#  'Buraxılış ili': '2007',
#  'Marka': 'LADA (VAZ)',
#  'Model': 'Niva',
#  'Mühərrik': '1.7 L',
#  'Mühərrikin gücü': '80 a.g.',
#  'Qiymət': '8 500 AZN',
#  'Rəng': 'Ağ',
#  'Sürətlər qutusu': 'Mexaniki',
#  'Yanacaq növü': 'Benzin',
#  'Yeni': 'Xeyr',
#  'Yürüş': '250 000 km',
#  'Ötürücü': 'Tam',
#  'Şəhər': 'Yardımlı'}

def rename_dict(dict_obj):
    dict_obj['ban_type'] = dict_obj.pop('Ban növü')
    return 

In [101]:
update_query = """
update cars set price = %s, marka = %s, model = %s, city = %s, 
year = %s where id = %s
"""

In [106]:
from pprint import pprint as pp
for item in items:
    pp(item)
    print('-----')
    url = item['url']
    response = requests.get(url)
    
    if response.status_code == 200:
        inner_bs = BeautifulSoup(response.content, 'lxml')
        props = inner_bs.find('ul', class_='product-properties')
        li_items = props.find_all('li')
#         print(li_items[0])

        md = dict()
    
        for li_item in li_items:
            key = li_item.find('label')
            value = li_item.find('div')
            if key:
                key = key.text
            if value:
                value = value.text
            if key:
                md[key] = value
                
        updated_values = (md['Qiymət'], 
                         md['Marka'],
                         md['Model'],
                         md['Şəhər'],
                         md['Buraxılış ili'],
                         item['id'])
        with connection.cursor() as cursor:
            cursor.execute(update_query, updated_values)
            connection.commit()
        
    else:
        print("Could not fetch url", url)
    

{'id': 1, 'url': 'https://turbo.az/autos/6368708-lada-vaz-niva'}
-----
{'id': 2, 'url': 'https://turbo.az/autos/6355461-opel-astra'}
-----
{'id': 3, 'url': 'https://turbo.az/autos/6368704-mercedes-cls-350'}
-----
{'id': 4, 'url': 'https://turbo.az/autos/6368703-lada-vaz-2115'}
-----
{'id': 5, 'url': 'https://turbo.az/autos/6312031-chevrolet-trax'}
-----
{'id': 6, 'url': 'https://turbo.az/autos/6357254-kia-optima'}
-----
{'id': 7, 'url': 'https://turbo.az/autos/6368701-daewoo-gentra'}
-----
{'id': 8, 'url': 'https://turbo.az/autos/6368700-opel-zafira'}
-----
{'id': 9, 'url': 'https://turbo.az/autos/6368712-toyota-rav-4'}
-----
{'id': 10, 'url': 'https://turbo.az/autos/6368699-ford-fusion'}
-----
{'id': 11, 'url': 'https://turbo.az/autos/6355765-mitsubishi-mirage'}
-----
{'id': 12, 'url': 'https://turbo.az/autos/6324514-bmw-320'}
-----
{'id': 13, 'url': 'https://turbo.az/autos/6360229-opel-astra'}
-----
{'id': 14, 'url': 'https://turbo.az/autos/6368682-iveco-astra-hd8'}
-----
{'id': 15, 

In [96]:
update_query = """
update cars set price = %s, marka = %s, model = %s, city = %s, 
year = %s where id = %s
"""


1. the same name

In [84]:
md = {'Ban növü': 'Offroader / SUV',
 'Buraxılış ili': '2007',
 'Marka': 'LADA (VAZ)',
 'Model': 'Niva',
 'Mühərrik': '1.7 L',
 'Mühərrikin gücü': '80 a.g.',
 'Qiymət': '8 500 AZN',
 'Rəng': 'Ağ',
 'Sürətlər qutusu': 'Mexaniki',
 'Yanacaq növü': 'Benzin',
 'Yeni': 'Xeyr',
 'Yürüş': '250 000 km',
 'Ötürücü': 'Tam',
 'Şəhər': 'Yardımlı'}

In [88]:
md.pop('Model')

KeyError: 'Model'

In [89]:
md['color'] = md.pop('Rəng')

In [91]:
md['city'] = md.get('Şəhər')

In [92]:
md

{'Ban növü': 'Offroader / SUV',
 'Buraxılış ili': '2007',
 'Marka': 'LADA (VAZ)',
 'Mühərrik': '1.7 L',
 'Mühərrikin gücü': '80 a.g.',
 'Qiymət': '8 500 AZN',
 'Sürətlər qutusu': 'Mexaniki',
 'Yanacaq növü': 'Benzin',
 'Yeni': 'Xeyr',
 'Yürüş': '250 000 km',
 'Ötürücü': 'Tam',
 'Şəhər': 'Yardımlı',
 'color': 'Ağ',
 'city': 'Yardımlı'}